In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.append(str(project_root))

from matplotlib.lines import Line2D
from ipywidgets import Layout, FloatSlider, Checkbox, Text, widgets, IntSlider, Dropdown

from analyze_potential import *

import io
import matplotlib.pyplot as plt
from PIL import Image
import win32clipboard

def copy_figure_to_clipboard(fig):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", dpi=300, bbox_inches="tight")
    buf.seek(0)

    image = Image.open(buf)
    output = io.BytesIO()
    image.convert("RGB").save(output, "BMP")
    data = output.getvalue()[14:]  # BMP header removed

    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(win32clipboard.CF_DIB, data)
    win32clipboard.CloseClipboard()

# Single system analysis:

In [3]:
def f(n_actual, dn, n_rays, unconcentricity, phi_max, desired_focus, T_c, diameter, plot, lens_type, copy_input_parameters, copy_cavity_parameters, eval_box, copy_image):

    if copy_input_parameters:
        copy_parameters_func(locals())

    n_actual, n_design, T_c, back_focal_length, R_1, R_2, R_2_signed, diameter = generate_input_parameters_for_lenses(lens_type, dn)

    defocus = choose_source_position_for_desired_focus_analytic(
        desired_focus=desired_focus,
        T_c=T_c,
        n_design=n_design,
        diameter=diameter,
        back_focal_length=back_focal_length,
        R_1=R_1,
        R_2=R_2_signed,
    )

    results_dict = analyze_potential_old(R_1=R_1, R_2=R_2_signed, back_focal_length=back_focal_length,
                                                         defocus=defocus, T_c=T_c, n_design=n_design, diameter=diameter,
                                                         unconcentricity=unconcentricity, n_actual=n_actual,
                                                         n_rays=n_rays, phi_max=phi_max, extract_R_analytically=True,
                                                         print_tests=False)

    if plot:
        # plt.close('all')
        fig, ax = plot_results(results_dict, far_away_plane=True, unconcentricity=unconcentricity)
        center = results_dict["center_of_curvature"]
        plt.suptitle(
            f"aspheric={aspheric}, desired_focus = {desired_focus:.3e}m, n_design: {n_design:.3f}, n_actual: {n_actual:.3f}, Lens focal length: {back_focal_length * 1e3:.1f} mm, Defocus: z_lens -> z_lens + {defocus * 1e3:.1f} mm, T_c: {T_c * 1e3:.1f} mm, Diameter: {diameter * 1e3:.2f} mm"
        )
        plt.show()

    if copy_cavity_parameters:
        pyperclip.copy(results_dict['cavity'].formatted_textual_params)

    if copy_image:
        copy_figure_to_clipboard(fig)
    
    if eval_box != '':
        try:
            exec(f"print({eval_box})")
        except (NameError, AttributeError) as e:
            print(f'invalid expression: {e}')

# rest of parameters
n_actual = 1.8
dn = 0
n_rays = 30
unconcentricity = 30e-4  # np.float64(0.007610344827586207)  # ,  np.float64(0.007268965517241379)
phi_max = 0.25
desired_focus = 200e-3
T_c = 4.35e-3
diameter = 12.7e-3
plot = True
aspheric = True
lens_type = 'aspheric - lab'

widgets.interact(
    f,
    n_actual=FloatSlider(value=n_actual, min=1.0, max=2.0, step=0.0001, description='n actual', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    dn=FloatSlider(value=dn, min=-0.2, max=0.2, step=0.0001, description='dn', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    n_rays=IntSlider(value=n_rays, min=2, max=2000, step=1, description='n rays', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    unconcentricity=FloatSlider(value=unconcentricity, min=0.0, max=100e-3, step=1e-6, description='unconcentricity (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    phi_max=FloatSlider(value=phi_max, min=0.01, max=1.0, step=0.001, description='phi max', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    desired_focus=FloatSlider(value=desired_focus, min=1e-3, max=1.0, step=1e-4, description='desired focus (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    T_c=FloatSlider(value=T_c, min=0.5e-3, max=10e-3, step=1e-5, description='T_c (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    diameter=FloatSlider(value=diameter, min=1e-3, max=25e-3, step=1e-5, description='diameter (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    plot=Checkbox(value=plot, description='Plot results', layout=Layout(width='300px'), style={'description_width': 'initial'}),
    lens_type=Dropdown(options=[("Existing Aspheric", 'aspheric - lab'), ('Avantier', 'avantier'), ('Spherical - f, n like labs aspheric', 'spherical - like labs aspheric'), ('Aspherical - f, n like Avantier', 'aspheric - like avantier')], value=lens_type, description="Collimation mode", style={'description_width': 'initial'}),
    copy_input_parameters=Checkbox(value=False, description='Copy input parameters', style={'description_width': 'initial'}),
    copy_cavity_parameters=Checkbox(value=False, description='Copy cavity parameters', style={'description_width': 'initial'}),
    eval_box=Text(value='', placeholder='Type a Python expression to print (e.g., cavity.arms[0].length)', description='Evaluate:', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    copy_image=Checkbox(value=False, description='Copy image', style={'description_width': 'initial'}),

);

interactive(children=(FloatSlider(value=1.8, description='n actual', layout=Layout(width='1500px'), max=2.0, m…

# Multiple Unconcentricities analysis:

In [4]:
def f(maximal_unconcentricitiy, dn, n_rays, phi_max, desired_focus, lens_type, copy_input_parameters, copy_image):  # , eval_box
    if copy_input_parameters:
        copy_parameters_func(locals())

    n_actual, n_design, T_c, back_focal_length, R_1, R_2, R_2_signed, diameter = generate_input_parameters_for_lenses(lens_type, dn)
    defocus = choose_source_position_for_desired_focus_analytic(
    desired_focus=desired_focus,
    T_c=T_c,
    n_design=n_design,
    diameter=diameter,
    back_focal_length=back_focal_length,
    R_1=R_1,
    R_2=R_2_signed,
    )
    unconcentricities = np.linspace(maximal_unconcentricitiy, 0.1e-3, 30)
    paraxial_spot_sizes = np.zeros_like(unconcentricities)
    spot_size_boundaries = np.zeros_like(unconcentricities)
    paraxial_NAs = np.zeros_like(unconcentricities)
    left_NAs = np.zeros_like(unconcentricities)
    for i, u in enumerate(unconcentricities):
        results_dict = analyze_potential_old(R_1=R_1, R_2=R_2_signed,
                                                             back_focal_length=back_focal_length, defocus=defocus,
                                                             T_c=T_c, n_design=n_design, diameter=diameter,
                                                             unconcentricity=u, n_actual=n_actual, n_rays=n_rays,
                                                             phi_max=phi_max, extract_R_analytically=True,
                                                             print_tests=False)
        paraxial_spot_sizes[i] = results_dict["spot_size_paraxial"]
        paraxial_NAs[i] = results_dict["NA_paraxial"]
        left_NAs[i] = results_dict["cavity"].arms[0].mode_parameters.NA[0]
        try:
            spot_size_boundaries[i] = np.abs(results_dict["zero_derivative_points"])
        except TypeError:
            spot_size_boundaries[i] = np.nan  # If zero_derivative_points is None
    # %%
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.scatter(unconcentricities * 1e3, paraxial_spot_sizes * 1e3, label="Paraxial spot size", color="blue")
    ax.scatter(
        unconcentricities * 1e3, spot_size_boundaries * 1e3, label="Boundary of 2nd vs 4th order dominance", color="red"
    )
    ax.set_xlabel("Unconcentricity (mm)")
    ax.set_ylabel("Spot size / boundary (mm)")
    ax.set_title(f"paraxial spot size vs. aberrations limit\nLens type:={lens_type}, Concentric long arm length: {2 * desired_focus:.2e}")
    ax.grid()

    # twin y-axis for paraxial NAs
    ax2 = ax.twinx()
    ax2.plot(unconcentricities * 1e3, paraxial_NAs, label="Right NA", color="orange", linestyle="--")
    ax2.plot(unconcentricities * 1e3, left_NAs, label="Left NA", color="green", linestyle="--")
    ax2.set_ylabel("Paraxial NA (unitless)")

    # combined legend
    handles1, labels1 = ax.get_legend_handles_labels()
    handles2, labels2 = ax2.get_legend_handles_labels()
    ax.legend(handles1 + handles2, labels1 + labels2, loc="best")

    if copy_image:
        copy_figure_to_clipboard(fig)
    plt.show()
    # if eval_box != '':
    #     try:
    #         exec(f"print({eval_box})")
    #     except (NameError, AttributeError) as e:
    #         print(f'invalid expression: {e}')

# rest of parameters
n_actual = 1.8
dn = 0
n_rays = 500
unconcentricity = 30e-4  # np.float64(0.007610344827586207)  # ,  np.float64(0.007268965517241379)
phi_max = 0.25
desired_focus = 200e-3
T_c = 4.35e-3
diameter = 12.7e-3
plot = True
aspheric = True
lens_type ='aspheric - lab'

widgets.interact(
    f,
    n_actual=FloatSlider(value=n_actual, min=1.0, max=2.0, step=0.0001, description='n actual', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    maximal_unconcentricitiy=FloatSlider(value=10e-3, min=1e-3, max=200e-3, step=0.0001, description='Maximal Unconcentricity', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    dn=FloatSlider(value=dn, min=-0.2, max=0.2, step=0.0001, description='dn', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    n_rays=IntSlider(value=n_rays, min=2, max=2000, step=1, description='n rays', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    phi_max=FloatSlider(value=phi_max, min=0.01, max=1.0, step=0.001, description='phi max', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    desired_focus=FloatSlider(value=desired_focus, min=1e-3, max=1.0, step=1e-4, description='desired focus (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    T_c=FloatSlider(value=T_c, min=0.5e-3, max=10e-3, step=1e-5, description='T_c (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    diameter=FloatSlider(value=diameter, min=1e-3, max=25e-3, step=1e-5, description='diameter (m)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    lens_type=Dropdown(options=[("Existing Aspheric", 'aspheric - lab'), ('Avantier', 'avantier'), ('Spherical - f, n like labs aspheric', 'spherical - like labs aspheric'), ('Aspherical - f, n like Avantier', 'aspheric - like avantier')], value=lens_type, description="Collimation mode", style={'description_width': 'initial'}),
    copy_input_parameters=Checkbox(value=False, description='Copy input parameters', style={'description_width': 'initial'}),
    copy_image=Checkbox(value=False, description='Copy image', style={'description_width': 'initial'}),
    # eval_box=Text(value='', placeholder='Type a Python expression to print (e.g., cavity.arms[0].length)', description='Evaluate:', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
);

interactive(children=(FloatSlider(value=0.01, description='Maximal Unconcentricity', layout=Layout(width='1500…

# Two lenses cavity, single unconcentricity:

In [2]:
def f(
        n_actual_aspheric,
        n_rays,
        copy_image,
        unconcentricity,
        phi_max,
        defocus,
        back_focal_length_aspheric,
        T_c_aspheric,
        n_design_aspheric,
        n_design_spherical,
        n_actual_spherical,
        T_c_spherical,
        f_spherical,
        diameter,
):
    optical_system = generate_two_lenses_optical_system(defocus=defocus, back_focal_length_aspheric=back_focal_length_aspheric, T_c_aspheric=T_c_aspheric, n_design_aspheric=n_design_aspheric, n_actual_aspheric=n_actual_aspheric, n_design_spherical=n_design_spherical, n_actual_spherical=n_actual_spherical, T_c_spherical=T_c_spherical, f_spherical=f_spherical, diameter=diameter,)
    rays_0 = initialize_rays(defocus=defocus, n_rays=n_rays, phi_max=phi_max)
    results_dict = analyze_potential(optical_system=optical_system, rays_0=rays_0, unconcentricity=unconcentricity, print_tests=False, )
    fig, ax = plot_results(results_dict=results_dict, far_away_plane=True, unconcentricity=unconcentricity, potential_x_axis_angles=False, )

    if copy_image:
        copy_figure_to_clipboard(fig)

# rest of parameters
n_rays = 50
unconcentricity = 1e-3  # np.float64(0.007610344827586207)  # ,  np.float64(0.007268965517241379)
phi_max = 0.3
defocus = 0.0029887489470528557
back_focal_length_aspheric = 20e-3
T_c_aspheric = 4.35e-3
n_design_aspheric = 1.45
n_actual_aspheric = 1.45
n_design_spherical = 1.45
n_actual_spherical = 1.45
T_c_spherical = 4.35e-3
f_spherical = 100e-3
diameter = 12.7e-3

widgets.interact(
    f,
    n_actual_aspheric=FloatSlider(value=n_actual_aspheric, min=1.0, max=2.0, step=0.0001, description='n actual', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    n_rays=IntSlider(value=n_rays, min=2, max=2000, step=1, description='n rays', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    copy_image=Checkbox(value=False, description='Copy image', style={'description_width': 'initial'}),
    unconcentricity=FloatSlider(value=unconcentricity, min=-0.02, max=0.02, step=1e-5, description='unconcentricity', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    phi_max=FloatSlider(value=phi_max, min=0.0, max=1.0, step=0.001, description='phi max', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    defocus=FloatSlider(value=defocus, min=-0.01, max=0.01, step=1e-6, description='defocus', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    back_focal_length_aspheric=FloatSlider(value=back_focal_length_aspheric, min=1e-3, max=0.1, step=1e-4, description='back focal length (aspheric)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    T_c_aspheric=FloatSlider(value=T_c_aspheric, min=0.0, max=0.02, step=1e-5, description='T_c aspheric', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    n_design_aspheric=FloatSlider(value=n_design_aspheric, min=1.0, max=2.0, step=0.0001, description='n design (aspheric)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    n_design_spherical=FloatSlider(value=n_design_spherical, min=1.0, max=2.0, step=0.0001, description='n design (spherical)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    n_actual_spherical=FloatSlider(value=n_actual_spherical, min=1.0, max=2.0, step=0.0001, description='n actual (spherical)', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    T_c_spherical=FloatSlider(value=T_c_spherical, min=0.0, max=0.02, step=1e-5, description='T_c spherical', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    f_spherical=FloatSlider(value=f_spherical, min=1e-3, max=0.5, step=1e-4, description='f spherical', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
    diameter=FloatSlider(value=diameter, min=1e-3, max=0.05, step=1e-4, description='diameter', layout=Layout(width='1500px'), style={'description_width': 'initial'}),
);
plt.show()


interactive(children=(FloatSlider(value=1.45, description='n actual', layout=Layout(width='1500px'), max=2.0, …